In [1]:
require("readr")

data.path <- '/scratch/cj59/RAPA/'
out.path <- '/scratch/cj59/RAPA/shiny/'

wide.file.names <- list(
    expression="2021_RAPA_EXPRESSION.tsv.gz",
    denoised="2021_RAPA_DENOISED.tsv.gz",
    velocity="2021_RAPA_VELOCITY.tsv.gz",
    tfa_expression="2021_RAPA_TFA_EXPRESSION.tsv.gz",
    tfa_denoised="2021_RAPA_TFA_DENOISED.tsv.gz",
    tfa_velocity="2021_RAPA_TFA_VELOCITY.tsv.gz",
    tfa_decay_constant="2021_RAPA_TFA_DECAY_20MIN.tsv.gz",
    tfa_decay_latent="2021_RAPA_TFA_DECAY_LATENT.tsv.gz"
)

agg.file.names <- list(
    rapamycin="2021_RAPA_AGGREGATE_RAPAMYCIN.tsv.gz",
    cell_cycle="2021_RAPA_AGGREGATE_CELL_CYCLE.tsv.gz"
)
    
metadata.file.names <- list(
    metadata="2021_RAPA_METADATA.tsv.gz",
    gene_metadata="2021_RAPA_GENE_METADATA.tsv.gz"
)

load.tsv <- function(x) {
    print(paste("Loading", x))
    y <- readr::read_tsv(file.path(data.path, x))
    
    if ('...1' %in% colnames(y)) {
       y['gene'] <- y['...1']
        y['...1'] <- NULL 
    }

    return(y)
}

Loading required package: readr



In [2]:
test.rds <- readRDS(file.path(out.path, "YEL009C.rds"))

In [6]:
test.rds$rapamycin[test.rds$rapamycin$dataset == "decay_constants", ]

-9.5,-8.5,-7.5,-6.5,-5.5,-4.5,-3.5,-2.5,-1.5,-0.5,⋯,54.5,55.5,56.5,57.5,58.5,59.5,agg_func,dataset,experiment,gene
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
0.1155054,0.1030699,0.09664873,0.09945519,0.1168235,0.1153207,0.1285912,0.1277948,0.1336853,0.1152473,⋯,0.15397097,0.14901235,0.14200570,0.14538874,0.16507619,0.15087126,NA,decay_constants,1,YEL009C
0.1054430,0.0995140,0.09705604,0.08721061,0.1010539,0.1183023,0.1218369,0.1318656,0.1338618,0.1277203,⋯,0.04160186,0.04310948,0.04801858,0.04883974,0.05281252,0.05839893,NA,decay_constants,2,YEL009C
0.1171821,0.1118425,0.10939728,0.10059741,0.1095289,0.1178913,0.1250231,0.1303611,0.1336786,0.1222874,⋯,0.06201938,0.06192612,0.07066292,0.06609690,0.06465845,0.07169283,NA,decay_constants,ALL,YEL009C


In [ ]:
test.rds

In [ ]:
wide.data <- lapply(wide.file.names, load.tsv)
agg.data <- lapply(agg.file.names, load.tsv)
meta.data <- lapply(metadata.file.names, load.tsv)

In [ ]:
get.data <- function(gene.name) {
    list.obj <- list()
    
    for (dtype in names(wide.data)) {
        if (gene.name %in% colnames(wide.data[[dtype]])) {
            list.obj[[dtype]] <- wide.data[[dtype]][[gene.name]]
        }
    }
        
    for (dtype in names(agg.data)) {
        adata <- agg.data[[dtype]][agg.data[[dtype]][['gene']] == gene.name, ]
        
        if (nrow(adata) > 0) {
            list.obj[[dtype]] <- adata
        }
    }
    
    return(list.obj)
}

In [ ]:
pb <- txtProgressBar(
    min = 0,
    max = nrow(meta.data[['gene_metadata']]),
    initial = 0
) 

i <- 0

for (gene.name in meta.data[['gene_metadata']][['gene']]) {
    
    saveRDS(
        get.data(gene.name),
        file=file.path(out.path, paste0(gene.name, ".rds")),
        compress=FALSE
    )

    i <- i + 1
    setTxtProgressBar(pb,i)    
}

close(pb)